# 转置卷积transposed convolution  
对于像素级别的分割任务：传统的卷积操作、pooling操作会压缩画质，纯粹是不行的，如果输入和输出图像的空间维度相同，在以像素级分类的语义分割中将会很方便  
所谓的转置，就是输入中的更小的块-->去对应整个核张量，进行运算。
![转置卷积](../img/trans_conv.svg) . 
转置卷积的会产生多个中间张量，中间张量最终叠加成为输出张量，增加了w，h的数量

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

: 

: 

![转置卷积的维度计算](../img/Math_trans_conv.jpg) . 
一般转置卷积核是标准的正方形，转置卷积能够产生输出大于输入的图像卷积结果

In [ ]:
#我们可以对输入矩阵X和卷积核矩阵K实现基本的转置卷积运算trans_conv
def trans_conv(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] + h -1, X.shape[1] + w - 1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i: i + h, j: j + w] += X[i, j] * K
    return Y

In [ ]:
#简单验证一下结果
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X, K)

In [ ]:
#或者直接用高级API实现结果
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False) #输入输出的通道数
tconv.weight.data = K
tconv(X)

![调整步幅和填充](../img/trans_conv_stride2.svg)

In [ ]:
#填充、步幅、多通道
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
#填充为1实际上就是作用在输出上，把最外围挖掉了padding = 1，可以让输出变小
tconv.weight.data = K
tconv(X)
#同样的卷积核、步幅、填充，对于通道而言：卷积、转置卷积是一个相反的过程
X = torch.rand(size=(1, 10, 16, 16))
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3)
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)
tconv(conv(X)).shape == X.shape